In [35]:
import tensorflow.keras.backend as K
import multiprocessing
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
import os
#from tensorflow.keras.applications.imagenet_utils import preprocess_input

from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from skimage import img_as_float, img_as_ubyte
from skimage.io import imread, imsave
from skimage.transform import pyramid_gaussian, rescale

In [36]:
def show_img(image):
    fig, ax = plt.subplots(figsize=(18, 15))
    ax.imshow(image)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
def preprocess_img(image):
    return preprocess_input(np.expand_dims(image.astype(K.floatx()), 0))

def postprocess_img(image):
    image[:, :, :, 0] += 103.939
    image[:, :, :, 1] += 116.779
    image[:, :, :, 2] += 123.68

In [37]:
path = r'C:\Users\olisu\Pictures\Saved Pictures'
file = 'hamburg_hgb.jpg'
img_file = os.path.join(path, file)

orig_image_array = imread(img_file)
orig_image_float = img_as_float(orig_image_array)
pyramid = list(pyramid_gaussian(orig_image_float, downscale = 2, max_layer = 5))

for i, image in enumerate(pyramid):
    pyramid[i] = img_as_ubyte(pyramid[i])
    print('Bild {}) Grösse: {}'.format(i, pyramid[i].shape))

Bild 0) Grösse: (1200, 1920, 3)
Bild 1) Grösse: (600, 960, 3)
Bild 2) Grösse: (300, 480, 3)
Bild 3) Grösse: (150, 240, 3)
Bild 4) Grösse: (75, 120, 3)
Bild 5) Grösse: (38, 60, 3)


In [38]:
model = VGG19(include_top = False, weights='imagenet')
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [39]:
LAYERS = {
    'block5_conv1': 0.001,
    'block5_conv2': 0.001,
}

In [40]:
loss_tensor = 0.0
image_l2_weight = 0.005

for layer, weight in LAYERS.items():
    loss_tensor += (-weight * K.sum(K.square(model.get_layer(layer).output)))

loss_tensor += image_l2_weight * K.sum(K.square(model.layers[0].input))
_loss_funct = K.function(inputs = [model.layers[0].input], outputs = [loss_tensor])

loss_gradient = K.gradients(loss = loss_tensor, variables = [model.layers[0].input])
_gradient_funct = K.function(inputs = [model.layers[0].input],
outputs = loss_gradient)

def dd_loss(x, shape):
    return _loss-funct([x.reshape(shape)])[0]

def dd_gradient(x, shape):
    return _gradient_funct([x.reshape(shape)])[0].flatten().astype(np.float4)

In [41]:
def dream_image(image, iterations = 2):
    bounds = np.array(shape = (image.flatten().shape[0], 2))
    bounds[:, 0] = -128.0
    bounds[:, 1] = 128
    
    x0 = image.flatten()
    
    result = minimize(fun = dd_loss,
                     x0 = x0,
                     args = list(image.shape),
                     jac = dd_gradient,
                     method = 'L-BFGS-B',
                     bounds = bounds,
                     options = {'maxiter' : iterations})
    return postprocess_img(np.copy(result.x.reshape(image.shape)))

In [42]:
processed_image = None

for i, image in enumerate(pyramid[::-1]):
    print('Traumpyramide: {}{}'.format(len(pyramid)-i, image.shape))
    
    if processed_image is None:
        processed_image = dream_image(preprocess_img(image))
    else:
        h, w = image.shape[0:2]
        ph, pw = processed_image.shape[0:2]
        rescaled_image = rescale(processed_image, order = 5,
                    scale = (float(h)/float(ph), float(w)/float(pw)))
        combined_image = img_as_ubyte((1.2 * img_as_float(image) +
                    0.8 * rescaled_image) / 2.0)
        processed_image = dream_image(preprocess_img(combined_image),
                                     iterations = 5)

print('Ausgeträumt!')

Traumpyramide: 6(38, 60, 3)


TypeError: array() missing required argument 'object' (pos 1)

In [43]:
show_image(processed_image)

NameError: name 'show_image' is not defined